This example shows how to scroll through a website using AgentQL with the [Google Colaboratory](https://colab.research.google.com/) environment.

Install [AgentQL](https://pypi.org/project/agentql/) library

In [ ]:
%pip install agentql

Install the Playwright dependency required by AgentQL.

In [ ]:
!playwright install chromium

You can [store](https://medium.com/@parthdasawant/how-to-use-secrets-in-google-colab-450c38e3ec75) keys in Google Colab's secrets.

In [ ]:
import os

from google.colab import userdata

os.environ["AGENTQL_API_KEY"]=userdata.get('AGENTQL_API_KEY')

Run AgentQL script. Unfortunately, you can't see the browser window directly in Google Colab like you would on your local machine. However, let's still interact with it and take a screen recording of the browser session to see what’s happening. 

Please note that an online environment like Google Colab supports **asynchronous version** of AgentQL.

In [ ]:
import logging
import random
import asyncio

import agentql
from playwright.async_api import async_playwright
from agentql.ext.playwright.async_api import Page
from IPython.display import HTML
from base64 import b64encode

logging.basicConfig(level=logging.DEBUG)
log = logging.getLogger(__name__)


async def key_press_end_scroll(page: Page):
    await page.keyboard.press("End")


async def mouse_wheel_scroll(page: Page):
    viewport_height, total_height, scroll_height = await page.evaluate(
        "() => [window.innerHeight, document.body.scrollHeight, window.scrollY]"
    )
    while scroll_height < total_height:
        scroll_height = scroll_height + viewport_height
        await page.mouse.wheel(delta_x=0, delta_y=viewport_height)
        asyncio.sleep(random.uniform(0.05, 0.1))


QUERY = """
{
    page_title
    post_headers[]
}
"""

async with async_playwright() as playwright, await playwright.chromium.launch() as browser:

    # Set up the video recording
    video_dir = os.path.abspath("videos")
    context = await browser.new_context(
      record_video_dir="videos/",
      record_video_size={"width": 1280, "height": 720}
    )
    page = await agentql.wrap_async(await context.new_page())
    
    log.info("Navigating to the page...")

    await page.goto("https://infinite-scroll.com/demo/full-page/")

    await page.wait_for_page_ready_state()

    num_extra_pages_to_load = 3

    for times in range(num_extra_pages_to_load):
        log.info(f"Scrolling to the bottom of the page... (num_times = {times+1})")
        await key_press_end_scroll(page)
        await page.wait_for_page_ready_state()
        log.info("Content loaded!")

    log.info("Issuing AgentQL data query...")
    response = await page.query_data(QUERY)

    log.info(f"AgentQL response: {response}")

    # Display the video
    video_files = [f for f in os.listdir(video_dir) if f.endswith('.webm')]
    if video_files:
        video_path = os.path.join(video_dir, video_files[0])
        with open(video_path, 'rb') as f:
          video_bytes = f.read()
            
        video_b64 = b64encode(video_bytes).decode('utf-8')
        video_html = f"""
        <video width="800" controls>
          <source src="data:video/webm;base64,{video_b64}" type="video/webm">
          Your browser does not support the video tag.
        </video>
        """
        display(HTML(video_html))
    else:
        print("No video file was created")     